<a href="https://colab.research.google.com/github/ClesteA/MedSim/blob/main/Untitled1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
import torch
major_version, minor_version = torch.cuda.get_device_capability()

# 1. Unsloth'u Colab için özel parametreyle kuruyoruz
!pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"

# 2. Xformers ve diğerlerini derleme yapmadan (no-deps) kuruyoruz
# Bu komut pip'in "dur ben bunu derleyeyim" demesini engeller.
!pip install --no-deps xformers trl peft accelerate bitsandbytes

# 3. Gereksiz uyarıları susturmak için ek paketler
!pip install --no-deps packaging ninja einops

In [2]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048 # Ne kadar uzun metin okuyabileceği
dtype = None # None yapınca otomatik algılar (Float16)
load_in_4bit = True # 4-bit quantization (Hız ve hafıza için şart)

# Llama 3.1 8B Instruct modelini yüklüyoruz
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Modele LoRA adaptörlerini ekliyoruz (Eğitilebilir hale getiriyoruz)
# Modeli yüklediğiniz kod bloğunda şu ayarları güncelleyin:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64, # ÖNCEKİ MUHTEMELEN 16 İDİ. Bunu 64 yapın (Kapasiteyi 4 katına çıkarır).
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 3407,
    use_rslora = False,
    loftq_config = None,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.4: Fast Llama patching. Transformers: 4.57.2.
   \\   /|    NVIDIA L4. Num GPUs = 1. Max memory: 22.161 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 8.9. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unsloth 2025.11.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [3]:
import json
from datasets import Dataset

# Dosya adını buraya yaz (Colab'e yüklediğin dosya)
DOSYA_ADI = "/content/medsim_genis_db_v2.json"

# Veriyi oku
try:
    with open(DOSYA_ADI, 'r', encoding='utf-8') as f:
        ham_veri = json.load(f)
    print(f"{len(ham_veri)} adet vaka bulundu. İşleniyor...")
except FileNotFoundError:
    print("HATA: Dosya bulunamadı! Lütfen sol menüden dosyayı yükleyin.")
    ham_veri = []

# Alpaca Prompt Formatı (Modelin instruction'ı anlaması için standart kalıp)
alpaca_prompt = """Aşağıda bir görevi tanımlayan bir talimat ve bağlam sağlayan bir girdi bulunmaktadır. İsteği uygun şekilde tamamlayan bir yanıt yazın.

### Talimat:
{}

### Girdi:
{}

### Yanıt:
{}"""

instruction_text = "Sen uzman bir dahiliye doktorusun. Aşağıdaki hasta bilgilerine dayanarak tanıyı ve bulguları JSON formatında analiz et. Ayrıca Türkçe dil yeteneklerini de geliştir."
EOS_TOKEN = tokenizer.eos_token

formatted_data = []

for vaka in ham_veri:
    # Girdi: Hasta bilgileri
    input_text = f"Hasta: {vaka['hasta_kimlik']['ad_soyad']}, {vaka['hasta_kimlik']['yas']} yaşında, {vaka['hasta_kimlik']['meslek']}.\nŞikayet: {vaka['hasta_kimlik']['sikayet']}"

    # Çıktı: Modelin öğrenmesini istediğimiz JSON cevabı
    output_text = json.dumps({
        "tani": vaka['gizli_tani'],
        "anamnez": vaka['anamnez'],
        "bulgular": vaka['bulgular']
    }, ensure_ascii=False)

    # Formatı birleştir
    text = alpaca_prompt.format(instruction_text, input_text, output_text) + EOS_TOKEN
    formatted_data.append({"text": text})

# HuggingFace Dataset formatına çevir
dataset = Dataset.from_list(formatted_data)
print("Veri seti eğitime hazır!")

513 adet vaka bulundu. İşleniyor...
Veri seti eğitime hazır!


In [4]:
from transformers import TrainerCallback, TrainingArguments # TrainingArguments eklendi
from trl import SFTTrainer # SFTTrainer eklendi
from unsloth import is_bfloat16_supported # is_bfloat16_supported eklendi

# --- 1. LOSS KONTROL MEKANİZMASI (FREN SİSTEMİ) ---
class TargetLossCallback(TrainerCallback):
    def on_log(self, args, state, control, logs=None, **kwargs): # **kwargs eklendi
        if logs and 'loss' in logs:
            current_loss = logs['loss']
            # Loss 0.38'in altına inerse tehlike çanları çalar, durdururuz.
            # 0.4 hedeflediğimiz için 0.38 güvenli alt sınırdır.
            if current_loss < 0.38:
                print(f"\n✋ HEDEF LOSS'A ULAŞILDI: {current_loss:.4f}. Ezberi önlemek için durduruluyor.")
                control.should_training_stop = True

# --- 2. TRAINER AYARLARI ---
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False,
    args = TrainingArguments(
        # Batch size'ı 2'de tutuyoruz ama grad. accum. düşürüyoruz.
        per_device_train_batch_size = 2,

        # HER 2 ADIMDA BİR GÜNCELLEME (Daha sıkı öğrenme)
        gradient_accumulation_steps = 2,

        # Bu ayarlarla 3 Epoch yetmeyebilir, 5 Epoch yapalım.
        # Callback olduğu için erken biterse sorun yok.
        num_train_epochs = 5,

        warmup_steps = 10,
        learning_rate = 2e-4, # Hızı koruyoruz
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(),
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
    ),
    callbacks=[TargetLossCallback()] # 0.38 - 0.40 hedefli callback aktif kalsın
)

print(f"Toplam Vaka: 513")
print(f"Bir adımda görülen vaka: {2*8} (Batch x GradAccum)")
print(f"Tüm veriyi 1 kez görmek için gereken adım: ~32")
print("Hedef: Tüm veriyi gör, Loss ~0.4 olunca dur.\n")

trainer_stats = trainer.train()

Unsloth: Tokenizing ["text"] (num_proc=16):   0%|          | 0/513 [00:00<?, ? examples/s]

The model is already on multiple devices. Skipping the move to device specified in `args`.


Toplam Vaka: 513
Bir adımda görülen vaka: 16 (Batch x GradAccum)
Tüm veriyi 1 kez görmek için gereken adım: ~32
Hedef: Tüm veriyi gör, Loss ~0.4 olunca dur.



==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 513 | Num Epochs = 5 | Total steps = 645
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 2
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 2 x 1) = 4
 "-____-"     Trainable parameters = 167,772,160 of 8,198,033,408 (2.05% trained)
wandb: Currently logged in as: buraktalha81 (buraktalha81-karadeniz-technical-university) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,1.958100
2,1.957900
3,1.975000
4,1.875800
5,2.002500
6,1.860800
7,1.773400
8,1.714600
9,1.755800
10,1.636200



✋ HEDEF LOSS'A ULAŞILDI: 0.3570. Ezberi önlemek için durduruluyor.


train/epoch,▁▁▁▁▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇████
train/global_step,▁▁▁▁▁▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇█████
train/grad_norm,▁▁▂▁▃▃▂▂▂▃▂▂▂▂▂▃▃▂▂▂▃▃▃▃▃▂▂▂▂▂▂▁▂▂▂▂▂▂▂█
train/learning_rate,▁▂████████████▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▆▆▆▆▆▆▆▆▆
train/loss,███▄▅▄▄▄▃▃▃▃▃▂▂▂▃▂▂▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▃▁
total_flos,2.368257673243853e+16
train/epoch,1.03113
train/global_step,133
train/grad_norm,0.27574
train/learning_rate,0.00016
train/loss,0.357


In [6]:
# Modeli GGUF formatında kaydet (q4_k_m = 4-bit orta kalite, dengeli)
print("Model GGUF formatına çevriliyor. Bu işlem birkaç dakika sürebilir...")
model.save_pretrained_gguf("model_medsim_tr", tokenizer, quantization_method = "q4_k_m")

print("Dönüştürme tamamlandı!")
print("Şimdi sol menüde 'model_medsim_tr' klasörü içindeki .gguf dosyasını indirebilirsin.")

Model GGUF formatına çevriliyor. Bu işlem birkaç dakika sürebilir...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:00<00:00, 14488.10it/s]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [00:58<00:00, 14.50s/it]


Unsloth: Merge process complete. Saved to `/content/model_medsim_tr`
Unsloth: Converting to GGUF format...
==((====))==  Unsloth: Conversion from HF to GGUF information
   \\   /|    [0] Installing llama.cpp might take 3 minutes.
O^O/ \_/ \    [1] Converting HF to GGUF bf16 might take 3 minutes.
\        /    [2] Converting GGUF bf16 to ['q4_k_m'] might take 10 minutes each.
 "-____-"     In total, you will have to wait at least 16 minutes.

Unsloth: llama.cpp found in the system. Skipping installation.
Unsloth: Preparing converter script...
Unsloth: [1] Converting model into bf16 GGUF format.
This might take 3 minutes...


RuntimeError: Unsloth: GGUF conversion failed: Unsloth: Failed to convert model to GGUF: Command 'python llama.cpp/unsloth_convert_hf_to_gguf.py --outfile Meta-Llama-3.1-8B-Instruct.BF16.gguf --outtype bf16 --split-max-size 50G model_medsim_tr' returned non-zero exit status 1.

In [7]:
# 1. Önce sadece eğitilmiş katmanları (LoRA) kaydet.
# Bu işlem çok hızlıdır ve hata verme ihtimali düşüktür.
model.save_pretrained("medsim_vaka_lora_backup")
tokenizer.save_pretrained("medsim_vaka_lora_backup")
print("✅ Güvenlik yedeği alındı! Eğitiminiz güvende.")

✅ Güvenlik yedeği alındı! Eğitiminiz güvende.


In [8]:
# 2. GGUF Dönüştürme
import os

print("GGUF dönüşümü başlatılıyor (q4_k_m)...")

try:
    # Modelin 16-bit halini GGUF'a çevirmeyi dene
    model.save_pretrained_gguf(
        "model_medsim_final",
        tokenizer,
        quantization_method = "q4_k_m"
    )
    print("✅ GGUF Dönüşümü BAŞARILI!")
    print("Dosya yolu: content/model_medsim_final/model-unsloth.gguf")

except RuntimeError as e:
    print(f"❌ GGUF Hatası oluştu: {e}")
    print("Muhtemel sebep: RAM yetmedi veya llama.cpp derlenemedi.")

    # Alternatif: Merged 16-bit (GGUF değil ama tam model) kaydet
    print("⚠️ Alternatif plan devreye giriyor: Model 16-bit (Merged) olarak kaydediliyor...")
    model.save_pretrained_merged(
        "model_medsim_16bit",
        tokenizer,
        save_method = "merged_16bit",
    )
    print("✅ Model 16-bit olarak kaydedildi! Bunu bilgisayarına indirip LM Studio ile kendin de GGUF yapabilirsin.")

GGUF dönüşümü başlatılıyor (q4_k_m)...
Unsloth: Merging model weights to 16-bit format...
Found HuggingFace hub cache directory: /root/.cache/huggingface/hub


model.safetensors.index.json: 0.00B [00:00, ?B/s]

Checking cache directory for required files...
Cache check failed: model-00001-of-00004.safetensors not found in local cache.
Not all required files found in cache. Will proceed with downloading.
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.


Unsloth: Preparing safetensor model files:   0%|          | 0/4 [00:00<?, ?it/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

...

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Unsloth: Preparing safetensor model files: 100%|██████████| 4/4 [00:49<00:00, 12.48s/it]


Note: tokenizer.model not found (this is OK for non-SentencePiece models)


Unsloth: Merging weights into 16bit: 100%|██████████| 4/4 [01:36<00:00, 24.16s/it]


Unsloth: Merge process complete. Saved to `/content/model_medsim_16bit`
✅ Model 16-bit olarak kaydedildi! Bunu bilgisayarına indirip LM Studio ile kendin de GGUF yapabilirsin.


In [9]:
import shutil

# Klasör adı (Alternatif yöntemle kaydettiğimiz isim)
klasor_adi = "model_medsim_16bit"
cikis_dosyasi = "medsim_doktor_yedek"

print(f"'{klasor_adi}' klasörü zipleniyor... Lütfen bekleyin.")

# Klasörü zip yap
shutil.make_archive(cikis_dosyasi, 'zip', klasor_adi)

print(f"✅ İŞLEM TAMAM! Sol menüde '{cikis_dosyasi}.zip' dosyasını göreceksiniz.")
print("O dosyaya sağ tıklayıp 'Download' diyerek bilgisayarınıza indirin.")

'model_medsim_16bit' klasörü zipleniyor... Lütfen bekleyin.


KeyboardInterrupt: 

In [11]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
import shutil
import os

source_dir = "/content/medsim_vaka_lora_backup" # Colab'deki model klasörün
destination_dir = "/content/drive/MyDrive/medsim_doktor_model_backup" # Drive'da oluşacak klasör

print(f"Transfer başlıyor... {source_dir} -> {destination_dir}")
print("Bu işlem dosya boyutuna göre 1-3 dakika sürebilir. Lütfen bekleyin...")

try:
    if os.path.exists(destination_dir):
        print("Uyarı: Drive'da bu isimde klasör zaten var, üzerine yazılıyor...")
        shutil.rmtree(destination_dir) # Varsa eskisini sil

    shutil.copytree(source_dir, destination_dir)
    print("✅ BAŞARILI! Modelin şu an Google Drive'ında güvende.")
    print("Drive'ına girip 'medsim_doktor_model' klasörünü kontrol edebilirsin.")

except Exception as e:
    print(f"❌ Bir hata oluştu: {e}")
    print("Olası sebep: Google Drive kotan dolmuş olabilir.")

Transfer başlıyor... /content/medsim_vaka_lora_backup -> /content/drive/MyDrive/medsim_doktor_model_backup
Bu işlem dosya boyutuna göre 1-3 dakika sürebilir. Lütfen bekleyin...
✅ BAŞARILI! Modelin şu an Google Drive'ında güvende.
Drive'ına girip 'medsim_doktor_model' klasörünü kontrol edebilirsin.
